## <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Using GenAI to postion our trucks based upon the weather

### License

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### Notebook Overview

- This notebook will use GenAI to determine the best places to locate our trucks based upon current weather events. 

- Notebook Logic:
    1. Create a row, per city, in the weather_gen_ai_insight table.  
        - For each city:
            - Query BigQuery for the current hourly weather forecast.
            - BigQuery will return the results as JSON.
            - Our LLM prompt will be created:
                - The prompt will ask for 5 waypoints to be created based upon the weather.
                - The prompt will ask for each truck to be tagged with a truck number so we can identify each truck in the results.
                - The prompt will ask for the time and address of where to move each truck. 
                - The prompt will ask the LLM to compute the latitude and longitude of each address.
                - The prompt will ask the LLM to provide an explaination of its thought process.
                - The prompt will provide example JSON results so the LLM knows the output format to generate the results.
                - The prompt will include the wather from BigQuery as the context.

            - A row will be inserted into the weather_gen_ai_insight table.

    2. An update statement will be run on the weather_gen_ai_insight table which will call GENERATE_TEXT (gemini-pro) to execute the LLM prompt.  
        - We save our prompt in the table so we can later inspect it or reprocess the prompt.

    3. The results of the GENERATE_TEXT are JSON so the inner JSON is then parsed into the generated_insight_text and generated_insight_json fields.

    4. The data is then displayed.

## Initialize Python

In [ ]:
# No need to set these
city_names=["New York City", "London", "Tokyo", "San Francisco"]
city_ids=[1,2,3,4]
city_temperature=["fahrenheit", "celsius", "celsius", "fahrenheit"]
number_of_coffee_trucks = "4"

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

## Supporting Functions

In [ ]:
def RunQuery(sql):
  import time

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      return False

In [ ]:
def GetNextPrimaryKey(fully_qualified_table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) AS result
    FROM `{fully_qualified_table_name}`
  """
  # print(sql)
  df_result = client.query(sql).to_dataframe()
  # display(df_result)
  return df_result['result'].iloc[0] + 1

## Create the GenAI Insights table

In [ ]:
%%bigquery
CREATE TABLE IF NOT EXISTS `${project_id}.${bigquery_data_beans_curated_dataset}.weather_gen_ai_insight`
--CREATE OR REPLACE TABLE `${project_id}.${bigquery_data_beans_curated_dataset}.weather_gen_ai_insight` -- only use to replace the entire table
(
    weather_gen_ai_insight_id   INTEGER   NOT NULL OPTIONS(description="Primary key."),
    weather_gen_ai_insight_type STRING    NOT NULL OPTIONS(description="The type of insight: WEATHER, EVENT, CAMPAIGN, SOCIAL, WAIT-TIME, SALES, VIDEO-AI"),
    insight_datetime            TIMESTAMP NOT NULL OPTIONS(description="The datetime of the GenAI insight."),
    applies_to_entity_type      STRING    NOT NULL OPTIONS(description="The type of entity to which the insight applies: Company, City, Truck, Customer"),
    applies_to_entity_id        INTEGER   NOT NULL OPTIONS(description="The id (primary key) of entity to which the insight applies."),
    applies_to_entity_name      STRING    NOT NULL OPTIONS(description="The name to which the insight applies."),
    llm_prompt                  STRING    NOT NULL OPTIONS(description="The LLM prompt."),
    ml_generate_json_result     JSON               OPTIONS(description="The raw JSON output of the LLM."),
    generated_insight_text      STRING             OPTIONS(description="The generated insight in text"),
    generated_insight_json      JSON               OPTIONS(description="The generated insight in JSON")
)
CLUSTER BY weather_gen_ai_insight_id;

In [ ]:
%%bigquery

-- Remove the current days items (so you can re-run)
DELETE
  FROM `${project_id}.${bigquery_data_beans_curated_dataset}.weather_gen_ai_insight`
 WHERE CAST(insight_datetime AS DATE) = CURRENT_DATE();

## Create the LLM prompt for each city and insert into or GenAI Insights table.

- You will need to run the Data-Bean-GenAI-Weather-Table notebook to download the current weather (for today)

In [ ]:
# Get the next primary key
weather_gen_ai_insight_id = GetNextPrimaryKey("${project_id}.${bigquery_data_beans_curated_dataset}.weather_gen_ai_insight","weather_gen_ai_insight_id")

# Loop for each city
for city_index in range(0, 4):
  print(f"Processing city: {city_ids[city_index]}")

  ##############################################################################
  # Get the weather data for the city and current date
  ##############################################################################
  sql=f"""WITH hourly_data AS
          (
            SELECT city_id, periods
              FROM `${project_id}.${bigquery_data_beans_curated_dataset}.weather`
                   CROSS JOIN UNNEST(JSON_QUERY_ARRAY(weather_json,'$.properties.periods')) AS periods
             WHERE city_id = {city_ids[city_index]}
               AND weather_date = CURRENT_DATE()
          )
          , hourly_fields AS
          (
            SELECT city_id,
                   CAST(SUBSTR(JSON_VALUE(periods, '$.startTime'),1,18)AS datetime) AS startTime,
                   CAST(SUBSTR(JSON_VALUE(periods, '$.endTime'),1,18)AS datetime) AS endTime,
                   ROUND(CAST(JSON_VALUE(periods, '$.temperature') AS FLOAT64),1) AS temperatureFahrenheit,
                   ROUND((32 - CAST(JSON_VALUE(periods, '$.temperature') AS FLOAT64)) * (5/9),1)  AS temperatureCelsius,  -- (32°F − 32) × 5/9 = 0°C
                   CAST(JSON_VALUE(periods, '$.probabilityOfPrecipitation.value') AS FLOAT64) AS probabilityOfPrecipitation,
                   CAST(JSON_VALUE(periods, '$.shortForecast') AS STRING) AS shortForecast
               FROM hourly_data
          )
          , data AS
          (
          SELECT startTime,
                 endTime,
                 CASE WHEN city_id IN (1,4) -- US Fahrenheit
                     THEN temperatureFahrenheit
                     ELSE temperatureCelsius
                 END AS temperature,
                 probabilityOfPrecipitation,
                 shortForecast
            FROM hourly_fields
          )

          SELECT TO_JSON(data) AS weather_json
            FROM data
          ORDER BY startTime;
          """

  # Run the SQL to get the weather for this city
  print(f"Gathering weather for: {city_names[city_index]}")
  weather_df = RunQuery(sql)

  # Loop through the dataframe (we want a JSON string for our prompt)
  weather_json = ""
  for index, row in weather_df.iterrows():
    weather_json = weather_json + row['weather_json'] + "\n"

  ##############################################################################
  # Get the weather id so we can link back to this record.
  ##############################################################################
  sql = f"""SELECT weather_id
              FROM `${project_id}.${bigquery_data_beans_curated_dataset}.weather`
             WHERE city_id = {city_ids[city_index]}
               AND weather_date = CURRENT_DATE()"""

  print(f"Gathering weather_id for: {city_names[city_index]}")
  weather_id_df = RunQuery(sql)

  weather_id = ""
  for index, row in weather_id_df.iterrows():
    weather_id = weather_id + str(row['weather_id'])

  print(f"weather_id: {weather_id}")

  ##############################################################################
  # Create the LLM prompt
  ##############################################################################

  # Example return JSON (without the python f"" formatting)
  json_example_data='[ ' + \
   '{ "weather_id": 1, "truck_number": 1, "waypoint_number": 1, "location_address": "Central Park", ' + \
     '"location_time": "2023-12-03 08:00:00", "latitude" : 40.736874, "longitude": -73.985394, ' + \
     '"explanation": "Due to rain you should position this under the central park coverings." }, ' + \
   '{ "weather_id": 1, "truck_number" : 1, "waypoint_number": 2, "location_address": "Hyde Park", ' + \
     '"location_time": "2023-12-03 13:00:00", "latitude" : 40.736874, "longitude": -73.985394, ' + \
     '"explanation": "The sunny weather will have lots of people in the park." }' + \
     ']'

  llm_prompt=f"""You need to return JSON in the below "JSON format".
You run a fleet of {number_of_coffee_trucks} coffee trucks in {city_names[city_index]}.
You want to maximize profits by picking the best locations, based upon the weather, in order to maximize profits.
Create a route with up to 5 waypoints for each truck based upon the below weather forecast.
The weather forecast temperature field is in {city_temperature[city_index]}.
Return the results in JSON with no special characters or formatting.
Number the trucks from 1 through {number_of_coffee_trucks} and place the result in the field "truck_number".
Compute the address of the location and place the result in the field "location_address".
Compute the time the truck should be moved to the location and place the result in the field "location_time".
Compute the latitude of the field "location_address" and place the result in the field "latitude".
Compute the longitude of the field "location_address" and place the result in the field "longitude".
Use the constant value of {weather_id} for the "weather_id" field.
Explain your logic and place the result in the field "explanation".

JSON format:
{json_example_data}

Context:
- When the probabilityOfPrecipitation is high the it is likely to be raining or snowing.
- If the weather is the same or close the same for several hours, we do not need to move the truck.  Only move the truck if it has been more than 4 hours or if the weather is changing.
- Consider locations that do not have a nearby coffee shop.  Take advanage of the ablity for a coffee truck to move locations.

Weather Forecast:
{weather_json}
"""

  print(f"Created LLM Prompt for: {city_names[city_index]}")

  # Insert new row
  sql=f"""INSERT INTO `${project_id}.${bigquery_data_beans_curated_dataset}.weather_gen_ai_insight`
  (weather_gen_ai_insight_id, weather_gen_ai_insight_type, insight_datetime, applies_to_entity_type, applies_to_entity_id, applies_to_entity_name, llm_prompt)
  VALUES({weather_gen_ai_insight_id}, 'WEATHER', CURRENT_TIMESTAMP(), 'CITY', {city_ids[city_index]}, '{city_names[city_index]}',\"\"\"{llm_prompt}\"\"\")
  """.replace("  ","")

  RunQuery(sql)

  # Get the next primary key
  weather_gen_ai_insight_id = weather_gen_ai_insight_id + 1

  print(f"Inserted prompt into weather_gen_ai_insight with weather_gen_ai_insight_id = {weather_gen_ai_insight_id}")

#### Sample Prompt with Weather JSON
```
You run a fleet of 4 coffee trucks in New York City.
You want to maximize profits by picking the best locations, based upon the weather, in order to maximize profits.
Create a route with up to 5 waypoints for each truck based upon the below weather forecast.
The weather forecast temperature field is in fahrenheit.
Return the results in JSON with no special characters or formatting.
Number the trucks from 1 through 4 and place the result in the field "truck_number".
Compute the address of the location and place the result in the field "location_address".
Compute the time the truck should be moved to the location and place the result in the field "location_time".
Compute the latitude of the field "location_address" and place the result in the field "latitude".
Compute the longitude of the field "location_address" and place the result in the field "longitude".
Use the constant value of 5 for the "weather_id" field.
Explain your logic and place the result in the field "explanation".

Example Return Data:
[ { "weather_id": 1, "truck_number": 1, "waypoint_number": 1, "location_address": "Central Park", "location_time": "2023-12-03 08:00:00", "latitude" : 40.736874, "longitude": -73.985394, "explanation": "Due to rain you should position this under the central park coverings." }, { "weather_id": 1, "truck_number" : 1, "waypoint_number": 2, "location_address": "Hyde Park", "location_time": "2023-12-03 13:00:00", "latitude" : 40.736874, "longitude": -73.985394, "explanation": "The sunny weather will have lots of people in the park." }]

Context:
- When the probabilityOfPrecipitation is high the it is likely to be raining or snowing.
- If the weather is the same or close the same for several hours, we do not need to move the truck.Only move the truck if it has been more than 4 hours or if the weather is changing.
- Consider locations that do not have a nearby coffee shop.Take advanage of the ablity for a coffee truck to move locations.

Weather Forecast:
{"endTime":"2024-03-14T10:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Sunny","startTime":"2024-03-14T09:00:00","temperature":50.0}
{"endTime":"2024-03-14T11:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Sunny","startTime":"2024-03-14T10:00:00","temperature":54.0}
{"endTime":"2024-03-14T12:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Sunny","startTime":"2024-03-14T11:00:00","temperature":59.0}
{"endTime":"2024-03-14T13:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Sunny","startTime":"2024-03-14T12:00:00","temperature":63.0}
{"endTime":"2024-03-14T14:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Sunny","startTime":"2024-03-14T13:00:00","temperature":65.0}
{"endTime":"2024-03-14T15:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Sunny","startTime":"2024-03-14T14:00:00","temperature":68.0}
{"endTime":"2024-03-14T16:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Sunny","startTime":"2024-03-14T15:00:00","temperature":69.0}
{"endTime":"2024-03-14T17:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Sunny","startTime":"2024-03-14T16:00:00","temperature":68.0}
{"endTime":"2024-03-14T18:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Sunny","startTime":"2024-03-14T17:00:00","temperature":67.0}
{"endTime":"2024-03-14T19:00:00","probabilityOfPrecipitation":2.0,"shortForecast":"Mostly Clear","startTime":"2024-03-14T18:00:00","temperature":64.0}
{"endTime":"2024-03-14T20:00:00","probabilityOfPrecipitation":2.0,"shortForecast":"Mostly Clear","startTime":"2024-03-14T19:00:00","temperature":63.0}
{"endTime":"2024-03-14T21:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Mostly Clear","startTime":"2024-03-14T20:00:00","temperature":61.0}
{"endTime":"2024-03-14T22:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-14T21:00:00","temperature":59.0}
{"endTime":"2024-03-14T23:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Mostly Cloudy","startTime":"2024-03-14T22:00:00","temperature":58.0}
{"endTime":"2024-03-15T00:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Mostly Cloudy","startTime":"2024-03-14T23:00:00","temperature":57.0}
{"endTime":"2024-03-15T01:00:00","probabilityOfPrecipitation":11.0,"shortForecast":"Mostly Cloudy","startTime":"2024-03-15T00:00:00","temperature":56.0}
{"endTime":"2024-03-15T02:00:00","probabilityOfPrecipitation":11.0,"shortForecast":"Mostly Cloudy","startTime":"2024-03-15T01:00:00","temperature":55.0}
{"endTime":"2024-03-15T03:00:00","probabilityOfPrecipitation":55.0,"shortForecast":"Rain Showers Likely","startTime":"2024-03-15T02:00:00","temperature":54.0}
{"endTime":"2024-03-15T04:00:00","probabilityOfPrecipitation":55.0,"shortForecast":"Rain Showers Likely","startTime":"2024-03-15T03:00:00","temperature":54.0}
{"endTime":"2024-03-15T05:00:00","probabilityOfPrecipitation":55.0,"shortForecast":"Rain Showers Likely","startTime":"2024-03-15T04:00:00","temperature":53.0}
{"endTime":"2024-03-15T06:00:00","probabilityOfPrecipitation":55.0,"shortForecast":"Rain Showers Likely","startTime":"2024-03-15T05:00:00","temperature":52.0}
{"endTime":"2024-03-15T07:00:00","probabilityOfPrecipitation":44.0,"shortForecast":"Chance Rain Showers","startTime":"2024-03-15T06:00:00","temperature":52.0}
{"endTime":"2024-03-15T08:00:00","probabilityOfPrecipitation":44.0,"shortForecast":"Chance Rain Showers","startTime":"2024-03-15T07:00:00","temperature":52.0}
{"endTime":"2024-03-15T09:00:00","probabilityOfPrecipitation":37.0,"shortForecast":"Chance Rain Showers","startTime":"2024-03-15T08:00:00","temperature":53.0}
{"endTime":"2024-03-15T10:00:00","probabilityOfPrecipitation":37.0,"shortForecast":"Chance Rain Showers","startTime":"2024-03-15T09:00:00","temperature":54.0}
{"endTime":"2024-03-15T11:00:00","probabilityOfPrecipitation":37.0,"shortForecast":"Chance Rain Showers","startTime":"2024-03-15T10:00:00","temperature":56.0}
{"endTime":"2024-03-15T12:00:00","probabilityOfPrecipitation":37.0,"shortForecast":"Chance Rain Showers","startTime":"2024-03-15T11:00:00","temperature":58.0}
{"endTime":"2024-03-15T13:00:00","probabilityOfPrecipitation":37.0,"shortForecast":"Chance Rain Showers","startTime":"2024-03-15T12:00:00","temperature":60.0}
{"endTime":"2024-03-15T14:00:00","probabilityOfPrecipitation":37.0,"shortForecast":"Chance Rain Showers","startTime":"2024-03-15T13:00:00","temperature":62.0}
{"endTime":"2024-03-15T15:00:00","probabilityOfPrecipitation":58.0,"shortForecast":"Rain Showers Likely","startTime":"2024-03-15T14:00:00","temperature":63.0}
{"endTime":"2024-03-15T16:00:00","probabilityOfPrecipitation":58.0,"shortForecast":"Rain Showers Likely","startTime":"2024-03-15T15:00:00","temperature":64.0}
{"endTime":"2024-03-15T17:00:00","probabilityOfPrecipitation":58.0,"shortForecast":"Rain Showers Likely","startTime":"2024-03-15T16:00:00","temperature":63.0}
{"endTime":"2024-03-15T18:00:00","probabilityOfPrecipitation":58.0,"shortForecast":"Rain Showers Likely","startTime":"2024-03-15T17:00:00","temperature":62.0}
{"endTime":"2024-03-15T19:00:00","probabilityOfPrecipitation":47.0,"shortForecast":"Chance Rain Showers","startTime":"2024-03-15T18:00:00","temperature":60.0}
{"endTime":"2024-03-15T20:00:00","probabilityOfPrecipitation":47.0,"shortForecast":"Chance Rain Showers","startTime":"2024-03-15T19:00:00","temperature":58.0}
{"endTime":"2024-03-15T21:00:00","probabilityOfPrecipitation":36.0,"shortForecast":"Chance Rain Showers","startTime":"2024-03-15T20:00:00","temperature":56.0}
{"endTime":"2024-03-15T22:00:00","probabilityOfPrecipitation":36.0,"shortForecast":"Chance Rain Showers","startTime":"2024-03-15T21:00:00","temperature":55.0}
{"endTime":"2024-03-15T23:00:00","probabilityOfPrecipitation":36.0,"shortForecast":"Chance Rain Showers","startTime":"2024-03-15T22:00:00","temperature":53.0}
{"endTime":"2024-03-16T00:00:00","probabilityOfPrecipitation":36.0,"shortForecast":"Chance Rain Showers","startTime":"2024-03-15T23:00:00","temperature":52.0}
{"endTime":"2024-03-16T01:00:00","probabilityOfPrecipitation":18.0,"shortForecast":"Slight Chance Rain Showers","startTime":"2024-03-16T00:00:00","temperature":50.0}
{"endTime":"2024-03-16T02:00:00","probabilityOfPrecipitation":18.0,"shortForecast":"Slight Chance Rain Showers","startTime":"2024-03-16T01:00:00","temperature":49.0}
{"endTime":"2024-03-16T03:00:00","probabilityOfPrecipitation":12.0,"shortForecast":"Mostly Cloudy","startTime":"2024-03-16T02:00:00","temperature":48.0}
{"endTime":"2024-03-16T04:00:00","probabilityOfPrecipitation":12.0,"shortForecast":"Mostly Cloudy","startTime":"2024-03-16T03:00:00","temperature":47.0}
{"endTime":"2024-03-16T05:00:00","probabilityOfPrecipitation":12.0,"shortForecast":"Mostly Cloudy","startTime":"2024-03-16T04:00:00","temperature":47.0}
{"endTime":"2024-03-16T06:00:00","probabilityOfPrecipitation":12.0,"shortForecast":"Mostly Cloudy","startTime":"2024-03-16T05:00:00","temperature":46.0}
{"endTime":"2024-03-16T07:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Partly Sunny","startTime":"2024-03-16T06:00:00","temperature":45.0}
{"endTime":"2024-03-16T08:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-16T07:00:00","temperature":45.0}
{"endTime":"2024-03-16T09:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-16T08:00:00","temperature":45.0}
{"endTime":"2024-03-16T10:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-16T09:00:00","temperature":45.0}
{"endTime":"2024-03-16T11:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-16T10:00:00","temperature":47.0}
{"endTime":"2024-03-16T12:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-16T11:00:00","temperature":50.0}
{"endTime":"2024-03-16T13:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-16T12:00:00","temperature":54.0}
{"endTime":"2024-03-16T14:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-16T13:00:00","temperature":57.0}
{"endTime":"2024-03-16T15:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-16T14:00:00","temperature":59.0}
{"endTime":"2024-03-16T16:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-16T15:00:00","temperature":58.0}
{"endTime":"2024-03-16T17:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-16T16:00:00","temperature":56.0}
{"endTime":"2024-03-16T18:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-16T17:00:00","temperature":54.0}
{"endTime":"2024-03-16T19:00:00","probabilityOfPrecipitation":2.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-16T18:00:00","temperature":52.0}
{"endTime":"2024-03-16T20:00:00","probabilityOfPrecipitation":2.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-16T19:00:00","temperature":51.0}
{"endTime":"2024-03-16T21:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-16T20:00:00","temperature":51.0}
{"endTime":"2024-03-16T22:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-16T21:00:00","temperature":51.0}
{"endTime":"2024-03-16T23:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-16T22:00:00","temperature":50.0}
{"endTime":"2024-03-17T00:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-16T23:00:00","temperature":50.0}
{"endTime":"2024-03-17T01:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-17T00:00:00","temperature":50.0}
{"endTime":"2024-03-17T02:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-17T01:00:00","temperature":50.0}
{"endTime":"2024-03-17T03:00:00","probabilityOfPrecipitation":10.0,"shortForecast":"Mostly Cloudy","startTime":"2024-03-17T02:00:00","temperature":49.0}
{"endTime":"2024-03-17T04:00:00","probabilityOfPrecipitation":10.0,"shortForecast":"Mostly Cloudy","startTime":"2024-03-17T03:00:00","temperature":49.0}
{"endTime":"2024-03-17T05:00:00","probabilityOfPrecipitation":10.0,"shortForecast":"Mostly Cloudy","startTime":"2024-03-17T04:00:00","temperature":48.0}
{"endTime":"2024-03-17T06:00:00","probabilityOfPrecipitation":10.0,"shortForecast":"Mostly Cloudy","startTime":"2024-03-17T05:00:00","temperature":48.0}
{"endTime":"2024-03-17T07:00:00","probabilityOfPrecipitation":10.0,"shortForecast":"Partly Sunny","startTime":"2024-03-17T06:00:00","temperature":48.0}
{"endTime":"2024-03-17T08:00:00","probabilityOfPrecipitation":10.0,"shortForecast":"Partly Sunny","startTime":"2024-03-17T07:00:00","temperature":48.0}
{"endTime":"2024-03-17T09:00:00","probabilityOfPrecipitation":19.0,"shortForecast":"Slight Chance Rain Showers","startTime":"2024-03-17T08:00:00","temperature":49.0}
{"endTime":"2024-03-17T10:00:00","probabilityOfPrecipitation":19.0,"shortForecast":"Slight Chance Rain Showers","startTime":"2024-03-17T09:00:00","temperature":50.0}
{"endTime":"2024-03-17T11:00:00","probabilityOfPrecipitation":19.0,"shortForecast":"Slight Chance Rain Showers","startTime":"2024-03-17T10:00:00","temperature":52.0}
{"endTime":"2024-03-17T12:00:00","probabilityOfPrecipitation":19.0,"shortForecast":"Slight Chance Rain Showers","startTime":"2024-03-17T11:00:00","temperature":54.0}
{"endTime":"2024-03-17T13:00:00","probabilityOfPrecipitation":19.0,"shortForecast":"Slight Chance Rain Showers","startTime":"2024-03-17T12:00:00","temperature":55.0}
{"endTime":"2024-03-17T14:00:00","probabilityOfPrecipitation":19.0,"shortForecast":"Slight Chance Rain Showers","startTime":"2024-03-17T13:00:00","temperature":56.0}
{"endTime":"2024-03-17T15:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Partly Sunny","startTime":"2024-03-17T14:00:00","temperature":57.0}
{"endTime":"2024-03-17T16:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Partly Sunny","startTime":"2024-03-17T15:00:00","temperature":57.0}
{"endTime":"2024-03-17T17:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Partly Sunny","startTime":"2024-03-17T16:00:00","temperature":57.0}
{"endTime":"2024-03-17T18:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-17T17:00:00","temperature":56.0}
{"endTime":"2024-03-17T19:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-17T18:00:00","temperature":55.0}
{"endTime":"2024-03-17T20:00:00","probabilityOfPrecipitation":5.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-17T19:00:00","temperature":54.0}
{"endTime":"2024-03-17T21:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-17T20:00:00","temperature":53.0}
{"endTime":"2024-03-17T22:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-17T21:00:00","temperature":52.0}
{"endTime":"2024-03-17T23:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-17T22:00:00","temperature":52.0}
{"endTime":"2024-03-18T00:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-17T23:00:00","temperature":51.0}
{"endTime":"2024-03-18T01:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-18T00:00:00","temperature":49.0}
{"endTime":"2024-03-18T02:00:00","probabilityOfPrecipitation":0.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-18T01:00:00","temperature":47.0}
{"endTime":"2024-03-18T03:00:00","probabilityOfPrecipitation":2.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-18T02:00:00","temperature":45.0}
{"endTime":"2024-03-18T04:00:00","probabilityOfPrecipitation":2.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-18T03:00:00","temperature":45.0}
{"endTime":"2024-03-18T05:00:00","probabilityOfPrecipitation":2.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-18T04:00:00","temperature":45.0}
{"endTime":"2024-03-18T06:00:00","probabilityOfPrecipitation":2.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-18T05:00:00","temperature":44.0}
{"endTime":"2024-03-18T07:00:00","probabilityOfPrecipitation":2.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-18T06:00:00","temperature":43.0}
{"endTime":"2024-03-18T08:00:00","probabilityOfPrecipitation":2.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-18T07:00:00","temperature":42.0}
{"endTime":"2024-03-18T09:00:00","probabilityOfPrecipitation":6.0,"shortForecast":"Partly Sunny","startTime":"2024-03-18T08:00:00","temperature":41.0}
{"endTime":"2024-03-18T10:00:00","probabilityOfPrecipitation":6.0,"shortForecast":"Partly Sunny","startTime":"2024-03-18T09:00:00","temperature":41.0}
{"endTime":"2024-03-18T11:00:00","probabilityOfPrecipitation":6.0,"shortForecast":"Partly Sunny","startTime":"2024-03-18T10:00:00","temperature":42.0}
{"endTime":"2024-03-18T12:00:00","probabilityOfPrecipitation":6.0,"shortForecast":"Partly Sunny","startTime":"2024-03-18T11:00:00","temperature":44.0}
{"endTime":"2024-03-18T13:00:00","probabilityOfPrecipitation":6.0,"shortForecast":"Partly Sunny","startTime":"2024-03-18T12:00:00","temperature":46.0}
{"endTime":"2024-03-18T14:00:00","probabilityOfPrecipitation":6.0,"shortForecast":"Partly Sunny","startTime":"2024-03-18T13:00:00","temperature":47.0}
{"endTime":"2024-03-18T15:00:00","probabilityOfPrecipitation":11.0,"shortForecast":"Partly Sunny","startTime":"2024-03-18T14:00:00","temperature":48.0}
{"endTime":"2024-03-18T16:00:00","probabilityOfPrecipitation":11.0,"shortForecast":"Partly Sunny","startTime":"2024-03-18T15:00:00","temperature":48.0}
{"endTime":"2024-03-18T17:00:00","probabilityOfPrecipitation":11.0,"shortForecast":"Partly Sunny","startTime":"2024-03-18T16:00:00","temperature":48.0}
{"endTime":"2024-03-18T18:00:00","probabilityOfPrecipitation":11.0,"shortForecast":"Partly Sunny","startTime":"2024-03-18T17:00:00","temperature":47.0}
{"endTime":"2024-03-18T19:00:00","probabilityOfPrecipitation":11.0,"shortForecast":"Mostly Cloudy","startTime":"2024-03-18T18:00:00","temperature":46.0}
{"endTime":"2024-03-18T20:00:00","probabilityOfPrecipitation":11.0,"shortForecast":"Mostly Cloudy","startTime":"2024-03-18T19:00:00","temperature":44.0}
{"endTime":"2024-03-18T21:00:00","probabilityOfPrecipitation":8.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-18T20:00:00","temperature":44.0}
{"endTime":"2024-03-18T22:00:00","probabilityOfPrecipitation":8.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-18T21:00:00","temperature":43.0}
{"endTime":"2024-03-18T23:00:00","probabilityOfPrecipitation":8.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-18T22:00:00","temperature":43.0}
{"endTime":"2024-03-19T00:00:00","probabilityOfPrecipitation":8.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-18T23:00:00","temperature":42.0}
{"endTime":"2024-03-19T01:00:00","probabilityOfPrecipitation":8.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-19T00:00:00","temperature":41.0}
{"endTime":"2024-03-19T02:00:00","probabilityOfPrecipitation":8.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-19T01:00:00","temperature":40.0}
{"endTime":"2024-03-19T03:00:00","probabilityOfPrecipitation":8.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-19T02:00:00","temperature":38.0}
{"endTime":"2024-03-19T04:00:00","probabilityOfPrecipitation":8.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-19T03:00:00","temperature":37.0}
{"endTime":"2024-03-19T05:00:00","probabilityOfPrecipitation":8.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-19T04:00:00","temperature":37.0}
{"endTime":"2024-03-19T06:00:00","probabilityOfPrecipitation":8.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-19T05:00:00","temperature":36.0}
{"endTime":"2024-03-19T07:00:00","probabilityOfPrecipitation":8.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-19T06:00:00","temperature":36.0}
{"endTime":"2024-03-19T08:00:00","probabilityOfPrecipitation":8.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-19T07:00:00","temperature":36.0}
{"endTime":"2024-03-19T09:00:00","probabilityOfPrecipitation":6.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-19T08:00:00","temperature":36.0}
{"endTime":"2024-03-19T10:00:00","probabilityOfPrecipitation":6.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-19T09:00:00","temperature":36.0}
{"endTime":"2024-03-19T11:00:00","probabilityOfPrecipitation":6.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-19T10:00:00","temperature":37.0}
{"endTime":"2024-03-19T12:00:00","probabilityOfPrecipitation":6.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-19T11:00:00","temperature":39.0}
{"endTime":"2024-03-19T13:00:00","probabilityOfPrecipitation":6.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-19T12:00:00","temperature":41.0}
{"endTime":"2024-03-19T14:00:00","probabilityOfPrecipitation":6.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-19T13:00:00","temperature":43.0}
{"endTime":"2024-03-19T15:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-19T14:00:00","temperature":45.0}
{"endTime":"2024-03-19T16:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-19T15:00:00","temperature":45.0}
{"endTime":"2024-03-19T17:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-19T16:00:00","temperature":44.0}
{"endTime":"2024-03-19T18:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-19T17:00:00","temperature":44.0}
{"endTime":"2024-03-19T19:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-19T18:00:00","temperature":44.0}
{"endTime":"2024-03-19T20:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-19T19:00:00","temperature":44.0}
{"endTime":"2024-03-19T21:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-19T20:00:00","temperature":44.0}
{"endTime":"2024-03-19T22:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-19T21:00:00","temperature":43.0}
{"endTime":"2024-03-19T23:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-19T22:00:00","temperature":41.0}
{"endTime":"2024-03-20T00:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-19T23:00:00","temperature":40.0}
{"endTime":"2024-03-20T01:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-20T00:00:00","temperature":39.0}
{"endTime":"2024-03-20T02:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-20T01:00:00","temperature":38.0}
{"endTime":"2024-03-20T03:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-20T02:00:00","temperature":38.0}
{"endTime":"2024-03-20T04:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-20T03:00:00","temperature":37.0}
{"endTime":"2024-03-20T05:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-20T04:00:00","temperature":36.0}
{"endTime":"2024-03-20T06:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-20T05:00:00","temperature":35.0}
{"endTime":"2024-03-20T07:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-20T06:00:00","temperature":35.0}
{"endTime":"2024-03-20T08:00:00","probabilityOfPrecipitation":7.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-20T07:00:00","temperature":35.0}
{"endTime":"2024-03-20T09:00:00","probabilityOfPrecipitation":10.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-20T08:00:00","temperature":35.0}
{"endTime":"2024-03-20T10:00:00","probabilityOfPrecipitation":10.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-20T09:00:00","temperature":36.0}
{"endTime":"2024-03-20T11:00:00","probabilityOfPrecipitation":10.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-20T10:00:00","temperature":37.0}
{"endTime":"2024-03-20T12:00:00","probabilityOfPrecipitation":10.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-20T11:00:00","temperature":39.0}
{"endTime":"2024-03-20T13:00:00","probabilityOfPrecipitation":10.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-20T12:00:00","temperature":42.0}
{"endTime":"2024-03-20T14:00:00","probabilityOfPrecipitation":10.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-20T13:00:00","temperature":45.0}
{"endTime":"2024-03-20T15:00:00","probabilityOfPrecipitation":9.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-20T14:00:00","temperature":47.0}
{"endTime":"2024-03-20T16:00:00","probabilityOfPrecipitation":9.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-20T15:00:00","temperature":47.0}
{"endTime":"2024-03-20T17:00:00","probabilityOfPrecipitation":9.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-20T16:00:00","temperature":46.0}
{"endTime":"2024-03-20T18:00:00","probabilityOfPrecipitation":9.0,"shortForecast":"Mostly Sunny","startTime":"2024-03-20T17:00:00","temperature":45.0}
{"endTime":"2024-03-20T19:00:00","probabilityOfPrecipitation":9.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-20T18:00:00","temperature":45.0}
{"endTime":"2024-03-20T20:00:00","probabilityOfPrecipitation":9.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-20T19:00:00","temperature":46.0}
{"endTime":"2024-03-20T21:00:00","probabilityOfPrecipitation":6.0,"shortForecast":"Partly Cloudy","startTime":"2024-03-20T20:00:00","temperature":45.0}
```

## Process the LLM prompt with BigQuery

In [ ]:
# Run the LLM
temperature=.75
max_output_tokens=8192
top_p=.8
top_k=30

sql=f"""UPDATE `${project_id}.${bigquery_data_beans_curated_dataset}.weather_gen_ai_insight` AS weather_gen_ai_insight
           SET insight_datetime = CURRENT_TIMESTAMP(),
               ml_generate_json_result = llm_query.ml_generate_text_result
           FROM (SELECT *
                   FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_data_beans_curated_dataset}.gemini_pro`,
                       (SELECT weather_gen_ai_insight_id,
                               llm_prompt AS prompt
                         FROM `${project_id}.${bigquery_data_beans_curated_dataset}.weather_gen_ai_insight`
                         WHERE (ml_generate_json_result IS NULL
                                OR
                                JSON_VALUE(ml_generate_json_result, '$.candidates[0].content') IS NULL
                                )
                       ),
                       STRUCT(
                         {temperature} AS temperature,
                         {max_output_tokens} AS max_output_tokens,
                         {top_p} AS top_p,
                         {top_k} AS top_k
                         ))
           ) AS llm_query
         WHERE weather_gen_ai_insight.weather_gen_ai_insight_id = llm_query.weather_gen_ai_insight_id;
"""
RunQuery(sql)

## Parse and view the results

In [ ]:
%%bigquery

# Parse the LLM results into json fields
UPDATE `${project_id}.${bigquery_data_beans_curated_dataset}.weather_gen_ai_insight`
   SET generated_insight_text = `${project_id}.${bigquery_data_beans_curated_dataset}.gemini_pro_result_as_string`(ml_generate_json_result),
       generated_insight_json = `${project_id}.${bigquery_data_beans_curated_dataset}.gemini_pro_result_as_json`(ml_generate_json_result)
 WHERE generated_insight_text IS NULL
 --AND weather_gen_ai_insight_id = 2

In [ ]:
%%bigquery

# Display the results
SELECT weather_gen_ai_insight_id, generated_insight_text, generated_insight_json
  FROM `${project_id}.${bigquery_data_beans_curated_dataset}.weather_gen_ai_insight`
 WHERE CAST(insight_datetime AS DATE) = CURRENT_DATE()
 ORDER BY weather_gen_ai_insight_id

#### Sample GenAI Result

```
[
    {
        "explanation": "Due to rain you should position this under the central park coverings.",
        "latitude": 40.736874,
        "location_address": "Central Park",
        "location_time": "2023-12-03 08:00:00",
        "longitude": -73.985394,
        "truck_number": 1,
        "waypoint_number": 1,
        "weather_id": 5
    },
    {
        "explanation": "The sunny weather will have lots of people in the park.",
        "latitude": 40.736874,
        "location_address": "Hyde Park",
        "location_time": "2023-12-03 13:00:00",
        "longitude": -73.985394,
        "truck_number": 1,
        "waypoint_number": 2,
        "weather_id": 5
    },
    {
        "explanation": "Due to rain you should position this under the Washington Square Park arch.",
        "latitude": 40.73061,
        "location_address": "Washington Square Park",
        "location_time": "2023-12-03 08:00:00",
        "longitude": -73.996226,
        "truck_number": 2,
        "waypoint_number": 1,
        "weather_id": 5
    },
    {
        "explanation": "The sunny weather will have lots of people in the park.",
        "latitude": 40.734711,
        "location_address": "Union Square Park",
        "location_time": "2023-12-03 13:00:00",
        "longitude": -73.989566,
        "truck_number": 2,
        "waypoint_number": 2,
        "weather_id": 5
    },
    {
        "explanation": "Due to rain you should position this under the Madison Square Park arch.",
        "latitude": 40.74391,
        "location_address": "Madison Square Park",
        "location_time": "2023-12-03 08:00:00",
        "longitude": -73.988196,
        "truck_number": 3,
        "waypoint_number": 1,
        "weather_id": 5
    },
    {
        "explanation": "The sunny weather will have lots of people in the park.",
        "latitude": 40.753866,
        "location_address": "Bryant Park",
        "location_time": "2023-12-03 13:00:00",
        "longitude": -73.983711,
        "truck_number": 3,
        "waypoint_number": 2,
        "weather_id": 5
    },
    {
        "explanation": "Due to rain you should position this under the Columbus Circle coverings.",
        "latitude": 40.767943,
        "location_address": "Columbus Circle",
        "location_time": "2023-12-03 08:00:00",
        "longitude": -73.981323,
        "truck_number": 4,
        "waypoint_number": 1,
        "weather_id": 5
    },
    {
        "explanation": "The sunny weather will have lots of people in the area.",
        "latitude": 40.758895,
        "location_address": "Times Square",
        "location_time": "2023-12-03 13:00:00",
        "longitude": -73.98513,
        "truck_number": 4,
        "waypoint_number": 2,
        "weather_id": 5
    }
]
```

In [ ]:
%%bigquery

-- Display the results
WITH location_data AS
(
  SELECT city_id,
         location_id,
         ROW_NUMBER() OVER (PARTITION BY city_id ORDER BY location_id) AS truck_number,
    FROM `${project_id}.${bigquery_data_beans_curated_dataset}.location`
   ORDER BY location_id
)
SELECT weather.weather_id,
       weather.weather_date,
       location_data.location_id,
       weather_gen_ai_insight.applies_to_entity_name,
       CAST(JSON_VALUE(json_data.truck_number) AS INT) AS truck_number,
       CAST(JSON_VALUE(json_data.waypoint_number) AS INT) AS waypoint_number,
       CAST(JSON_VALUE(json_data.location_address) AS STRING) AS location_address,
       CAST(JSON_VALUE(json_data.location_time) AS DATETIME) AS location_time,
       CAST(JSON_VALUE(json_data.latitude) AS FLOAT64) AS latitude,
       CAST(JSON_VALUE(json_data.longitude) AS FLOAT64) AS longitude,
       CAST(JSON_VALUE(json_data.explanation) AS STRING) AS explanation,
  FROM `${project_id}.${bigquery_data_beans_curated_dataset}.weather_gen_ai_insight` AS weather_gen_ai_insight
        CROSS JOIN UNNEST(JSON_EXTRACT_ARRAY(weather_gen_ai_insight.generated_insight_json)) AS json_data
        INNER JOIN `${project_id}.${bigquery_data_beans_curated_dataset}.weather` AS weather
                ON CAST(JSON_VALUE(json_data,'$.weather_id') AS INT64) = weather.weather_id
               AND weather.weather_date = CURRENT_DATE()
               AND CAST(weather_gen_ai_insight.insight_datetime AS DATE) = CURRENT_DATE()
        INNER JOIN location_data
                ON CAST(JSON_VALUE(json_data.truck_number) AS INT) = location_data.truck_number
               AND location_data.city_id = weather_gen_ai_insight.applies_to_entity_id
  ORDER BY applies_to_entity_name, weather_id, truck_number